In [1]:
from ovm import OVM, OVMParameters
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import sys

In [8]:
# switch to normal draw / pgf rendering
def set_pgf(use_pgf: bool):
    if use_pgf:
        matplotlib.use("pgf")
        matplotlib.rcParams.update({
            "pgf.texsystem": "pdflatex",
            'font.family': 'serif',
            'text.usetex': True,
            'pgf.rcfonts': False,
        })
    else:
        matplotlib.use("MacOSX")
        matplotlib.rcParams.update({
            "pgf.texsystem": "pdflatex",
            'font.family': 'serif',
            'text.usetex': False,
            'pgf.rcfonts': False,
        })

# renders the current plot to a pgf file
def render(filename: str):
    plt.savefig("figures/"+filename)

# Set this value then run the cell in order to apply the changes
set_pgf(True)

In [ ]:
jam_threshold = 50
model = OVM()
parameters = OVMParameters(dt=0.1, t_max=1000.0, max_distance=4.0)

In [ ]:
max_speeds = np.arange(1, 20.001, 0.1)
jam_time = np.zeros(max_speeds.size)

for i in np.arange(0, max_speeds.size):
    print("Computing max speed " + str(max_speeds[i]))
    parameters.max_speed = max_speeds[i]
    model.set(parameters)
    model.integrate()
    model.evaluate_jam()
    idx = np.diff(model.jam_evaluation > jam_threshold, prepend=False)
    times = model.time[idx]
    jam_time[i] = times[0] if times.size > 0 else parameters.t_max

In [67]:
plt.plot(max_speeds, jam_time)
plt.xlabel("maximum velocity")
plt.ylabel("time to jam")

plt.show()
# render("ovm_speed_comparison_2.pgf")

In [3]:
jam_threshold = 50
parameters2 = OVMParameters(dt=0.01, t_max=1000.0, max_speed=4.0)
model2 = OVM()
model2.set(parameters2)

In [4]:
max_distances = np.arange(1.0, 10.001, 0.1)
jam_time = np.zeros(max_distances.size)
jams = np.zeros(shape=(max_distances.size, model2.iterations))

for i in np.arange(0, max_distances.size):
    sys.stdout.write("\rProgress " + str(int(i / max_distances.size * 100.0)) + "%")
    sys.stdout.flush()
    
    parameters2.max_distance = max_distances[i]
    model2.set(parameters2)
    model2.integrate()
    model2.evaluate_jam()
    jams[i,:] = model2.jam_evaluation
print("Done")

Progress 98%Done


In [5]:
for i in np.arange(0, max_distances.size):
    jam = jams[i,:]
    idx = np.diff(jam > jam_threshold, prepend=False)
    times = model2.time[idx]
    jam_time[i] = times[0] if times.size > 0 else parameters2.t_max

In [9]:
plt.plot(max_distances, jam_time)
plt.xlabel("maximum distance")
plt.ylabel("time to jam")

render("ovm_distance_comparison_2.pgf")